In [ ]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh -y

# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

# CheckPoint

In [ ]:
# Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/198530'
model_name = 'JuggernautXL.safetensors'

%cd $checkpoints
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Lora

In [ ]:
# Install a LoRA in permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

%cd /kaggle/working/ComfyUI/models/loras
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Launch With RemoteMoe

In [ ]:
# Starting with RemoteMoe

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python /kaggle/working/ComfyUI/main.py & ssh -R 80:127.0.0.1:8188 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Launch with Pinggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

In [ ]:
import requests

# Define the URLs for the files
file_urls = {
    'arcface.onnx': 'https://huggingface.co/camenduru/Arc2Face/resolve/main/antelopev2/arcface.onnx?download=true',
    'scrfd_10g_bnkps.onnx': 'https://huggingface.co/camenduru/Arc2Face/resolve/main/antelopev2/scrfd_10g_bnkps.onnx?download=true'
}

# Define the target directory
target_dir = '/kaggle/working/ComfyUI/models/antelopev2/'

# Download and save each file
for file_name, url in file_urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(target_dir, file_name), 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name}")


In [ ]:
import os
import requests

# Define the target directory and URLs
target_dir = '/kaggle/working/ComfyUI/models/arc2face_checkpoints'
file_urls = {
    'config.json': 'https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arc2face/config.json?download=true',
    'diffusion_pytorch_model.safetensors': 'https://huggingface.co/FoivosPar/Arc2Face/resolve/main/arc2face/diffusion_pytorch_model.safetensors?download=true'
}

# Create the directory if it does not exist
os.makedirs(target_dir, exist_ok=True)

# Verify directory creation
if not os.path.isdir(target_dir):
    print(f"Directory '{target_dir}' could not be created.")
else:
    print(f"Directory '{target_dir}' successfully created.")

# Print out current working directory and contents of the target directory
print(f"Current working directory: {os.getcwd()}")
print(f"Contents of '/kaggle/working/ComfyUI/models/': {os.listdir('/kaggle/working/ComfyUI/models/')}")
print(f"Contents of '{target_dir}/': {os.listdir(target_dir)}")

# Download and save each file
for file_name, url in file_urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(target_dir, file_name)
        try:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name} to {file_path}")
        except Exception as e:
            print(f"Failed to save {file_name}: {e}")
    else:
        print(f"Failed to download: {file_name} (Status code: {response.status_code})")



In [ ]:
import os
import requests

# Define the target directory and URLs
encoder_dir = '/kaggle/working/ComfyUI/models/arc2face_checkpoints/encoder'
file_urls = {
    'config.json': 'https://huggingface.co/FoivosPar/Arc2Face/resolve/main/encoder/config.json?download=true',
    'pytorch_model.bin': 'https://huggingface.co/FoivosPar/Arc2Face/resolve/main/encoder/pytorch_model.bin?download=true'
}

# Create the directory if it does not exist
os.makedirs(encoder_dir, exist_ok=True)

# Download and save each file
for file_name, url in file_urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(encoder_dir, file_name)
        try:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name} to {file_path}")
        except Exception as e:
            print(f"Failed to save {file_name}: {e}")
    else:
        print(f"Failed to download: {file_name} (Status code: {response.status_code})")
